<a href="https://colab.research.google.com/github/muraliRoyal1/venkatasiva_INFO5731_Fall2020/blob/master/In_Class_exercises_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In class exercise 10

The purpose of the exercise is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K means, 
DBSCAN,
Hierarchical clustering. 

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 
    

In [9]:
#importing necessary libraries
import pandas as pd


In [10]:
#load the data
mysample=pd.read_csv("/content/sample_data/amazon_reviews_1.csv")

In [11]:
mysample.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"5.0"" Cell Phones Unlocked Android 5.1 MTK6580 ...",JUNING,89.99,5,This phone works well as how it supposed to be...,4.0
1,"5.0"" Cell Phones Unlocked Android 5.1 MTK6580 ...",JUNING,89.99,5,The phone is an unlocked phone and the suppor...,1.0
2,"5.0"" Cell Phones Unlocked Android 5.1 MTK6580 ...",JUNING,89.99,1,Cheap-- I am sending it back..,4.0
3,"5.0"" Cell Phones Unlocked Android 5.1 MTK6580 ...",JUNING,89.99,5,I bought this to replace my samsung galaxy s3 ...,1.0
4,"5.0"" Cell Phones Unlocked Android 5.1 MTK6580 ...",JUNING,89.99,5,"My niece is 15 and so hard on phones, my siste...",4.0


In [12]:
from nltk.corpus import stopwords
import nltk
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
from sklearn.cluster import KMeans
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import metrics


In [14]:
import gensim
import scipy
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering

In [15]:

import nltk
nltk.download('stopwords')
mysample['CleanedReviews'] = mysample['Reviews'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
mysample['CleanedReviews'] = mysample['CleanedReviews'].str.replace('[^\w\s]','')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
stop = stopwords.words('english')
mysample['CleanedReviews'] = mysample['CleanedReviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
mysample['CleanedReviews'] = mysample['CleanedReviews'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [17]:

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(mysample['CleanedReviews'].values)

In [18]:
# Implementing K-MEANS CLUSTERING

no_clusters = [x for x in range(3, 10)]
squared_errors = []
for cluster in no_clusters:
    kmeans = KMeans(n_clusters = cluster).fit(tfidf_vectors)
    squared_errors.append(kmeans.inertia_)

     

In [20]:
optimal_clusters = np.argmin(squared_errors) + 2 
xy = (optimal_clusters, min(squared_errors))
print ("Optimal number of clusters: - ", optimal_clusters)
print ("loss for Optimal cluster: - ", min(squared_errors))

Optimal number of clusters: -  8
loss for Optimal cluster: -  412.35887679458534


In [22]:
optimal_k = 8
model = KMeans(n_clusters = optimal_k).fit(tfidf_vectors)
labels = model.labels_
cluster_centers = model.cluster_centers_
terms = tfidf_vectorizer.get_feature_names()
silhouette_score = metrics.silhouette_score(tfidf_vectors, labels, metric='euclidean')
print('The Silhouette Score is: ', silhouette_score)
mysample['TF-IDF Cluster Labels'] = labels
print(mysample.groupby(['TF-IDF Cluster Labels'])['Reviews'].count())
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
for i in range(optimal_k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :optimal_k]:
        print(' %s' % terms[ind], end='')
        print()

The Silhouette Score is:  0.09740502415252084
TF-IDF Cluster Labels
0    128
1     28
2     20
3     12
4    118
5     25
6     38
7    143
Name: Reviews, dtype: int64
Cluster 0: excelent
 working
 chinese
 bueno
 exelente
 like
 love
 item
Cluster 1: excellent
 product
 recommend
 thank
 arrived
 much
 condition
 seller
Cluster 2: junk
 worst
 bought
 60
 charging
 pas
 one
 even
Cluster 3: ok
 far
 quite
 wcdma
 people
 slow
 congratulection
 entering
Cluster 4: phone
 sim
 card
 screen
 work
 use
 review
 one
Cluster 5: excelente
 telefono
 muy
 bueno
 producto
 faster
 favorite
 favor
Cluster 6: good
 phone
 product
 buy
 mobile
 thanks
 waist
 work
Cluster 7: phone
 work
 great
 charge
 nice
 arrived
 waste
 money


In [23]:
# DBSCAN CLUSTERING
list_of_reviews=[]
for review in mysample['CleanedReviews'].values:
    list_of_reviews.append(review.split())
w2v_model=gensim.models.Word2Vec(list_of_reviews, size=100, workers=4)

In [24]:
review_vectors = []
for review in list_of_reviews:
    review_vec = np.zeros(100)
    count_words = 0
    for word in review:
        try:
            vec = w2v_model.wv[word]
            review_vec += vec
            count_words += 1
        except:
            pass
    review_vec /= count_words
    review_vectors.append(review_vec)
    
review_vectors = np.array(review_vectors)
review_vectors = np.nan_to_num(review_vectors)
minPts = 2 * 100

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [25]:

def lower_bound(nums, target): 
    l, r = 0, len(nums) - 1
    while l <= r: 
        mid = int(l + (r - l) / 2)
        if nums[mid] >= target:
            r = mid - 1
        else:
            l = mid + 1
    return l

In [26]:
def compute_200th_nearest_neighbour(x, data): # Returns the distance of 200th nearest neighbour.
    dists = []
    for val in data:
        dist = np.sum((x - val) **2 )
        if(len(dists) == 200 and dists[199] > dist): 
            l = int(lower_bound(dists, dist))
            if l < 200 and l >= 0 and dists[l] > dist:
                dists[l] = dist
        else:
            dists.append(dist)
            dists.sort()
    
    return dists[199]

In [27]:

two_hundreth_neighbour = []
for val in review_vectors[:1500]:
    two_hundreth_neighbour.append(compute_200th_nearest_neighbour(val, review_vectors[:1500]) )
two_hundreth_neighbour.sort()

In [28]:
from sklearn.cluster import DBSCAN
optimal_epsilon = 8
dbscan = DBSCAN(eps = optimal_epsilon, min_samples = 200).fit(review_vectors)
mysample['DBSCAN Cluster Labels'] = dbscan.labels_
print(mysample.groupby(['DBSCAN Cluster Labels'])['Reviews'].count())

DBSCAN Cluster Labels
0    512
Name: Reviews, dtype: int64


In [33]:
# HIERARCHICAL CLUSTERING
agg_model = AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='ward')
Agg = agg_model.fit_predict(review_vectors)

h_labels = agg_model.labels_
h_silhouette_score = metrics.silhouette_score(review_vectors, h_labels, metric='euclidean')
print('silhouette score:',h_silhouette_score)

silhouette score: 0.5950469910833276


In one paragraph, please compare K means, DBSCAN and Hierarchical clustering. 

In [ ]:
"""
from the above analysis:
DBSCAN showed best performance followed by k-means clustering amd hierarchial clustering with 97% scoring and 60% respectively.